In [ ]:
import os
from sklearn.model_selection import StratifiedKFold
import shutil

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis'

In [ ]:
images_dir = ROOT_DIR + '/data/train/images'
labels_dir = ROOT_DIR + '/data/train/labels'
output_dir = ROOT_DIR + '/outputs/kfold_yolo_11'


In [ ]:
print(images_dir)

/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/data/train/images


In [ ]:
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.jpeg')])
label_files = [f.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt') for f in image_files]

# Asignar clases (aunque sea solo una clase, la necesitamos para StratifiedKFold)
def get_class_from_label(label_path):
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            if lines:
                return int(lines[0].split()[0])  # primer ID de clase
    except:
        return -1  # si hay un error
    return -1

y = [get_class_from_label(os.path.join(labels_dir, label)) for label in label_files]

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(image_files, y)):
    fold_dir = os.path.join(output_dir, f'fold{fold}')
    os.makedirs(os.path.join(fold_dir, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(fold_dir, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(fold_dir, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(fold_dir, 'labels', 'val'), exist_ok=True)

    for idx in train_idx:
        img = image_files[idx]
        lbl = label_files[idx]
        shutil.copy(os.path.join(images_dir, img), os.path.join(fold_dir, 'images', 'train', img))
        shutil.copy(os.path.join(labels_dir, lbl), os.path.join(fold_dir, 'labels', 'train', lbl))

    for idx in val_idx:
        img = image_files[idx]
        lbl = label_files[idx]
        shutil.copy(os.path.join(images_dir, img), os.path.join(fold_dir, 'images', 'val', img))
        shutil.copy(os.path.join(labels_dir, lbl), os.path.join(fold_dir, 'labels', 'val', lbl))


In [ ]:
import os

base_path = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11'
n_splits = 10  # número de folds

for fold in range(n_splits):
    fold_path = f"{base_path}/fold{fold}"
    yaml_content = f"""path: '{fold_path}'
train: images/train
val: images/val

# Classes
names:
  0: ladybug

"""

    yaml_path = os.path.join(fold_path, f'data_fold{fold}.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    print(f"✅ YAML generado: {yaml_path}")


✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/data_fold0.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold1/data_fold1.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold2/data_fold2.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold3/data_fold3.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold4/data_fold4.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold5/data_fold5.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold6/data_fold6.yaml
✅ YAML generado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold7/data_fold7.yaml
✅ YAML generado: /content/drive/

Probar con los modelos del fold a ver cual da el mejor rsultado en el test externo. Promediar la curva de loss promedio de los folds. Hacer la prueba de wilcoxon, usar paquete de python. Tomar los promedios, el mejor promedio, pivot estadistico en map50, usar map50, precision y recall y el u-test para la tabla del wilcoxon. Del loss usar solo un grafico del mejor modelo.
